In [1]:
import os
import re

import matplotlib.pyplot as plt
from utils.evaluations_utils import *
import numpy as np
%matplotlib inline
import matplotlib
import seaborn as sn
from create_entropy_estimation import EntropyEstimation
import re
from typing import List,Iterable

class CTWHandler():

    def __init__(self):
        self.__data=dict()
    def add_data(self,path:Iterable[str]):
        # data:List[cee.EntropyEstimation]=[]
        for i in path:
            data=EntropyEstimation.load(i)
            if data.tag not in self.__data:
                self.__data[data.tag]=dict()
            if data.file_name not in self.__data[data.tag]:
                self.__data[data.tag][data.file_name]=dict()
            self.__data[data.tag][data.file_name][data.file_index]=data

    def get_item_by_file_name_and_index(self,f_name,index):
        return {k:v[f_name][index] for k,v in self.__data.items()}

    def __iter__(self):
        for kt,vt in self.__data.items():
            for kf,vf in vt.items():
                for ki,vi in vf.items():
                    return dict(tag=kt,file=kf,index=ki,entropy=vi.entopy,n_spikes=np.sum(vi.s),l_segemnt=vi.s.size)

models={}
regex_match = re.compile('Rat.+CTW')
gmax_regex = re.compile('.*_([0-9](?:-[0-9])?(?:_noNMDA)?)_.*')
gmax_value_regex = re.compile('([0-9](?:\.[0-9])?)[a-z,A-Z]*')
models_pathes=[]
for i in os.listdir('entropy_data'):
    cur_dir_path=os.path.join('entropy_data',i)
    if regex_match.match(i) and os.path.isdir(cur_dir_path):
        for j in os.listdir(cur_dir_path):
            models_pathes.append(os.path.join(cur_dir_path,j))
        models[i] = gmax_regex.match(i).group(1).replace('-','.')
        models[i] = models[i].replace('_noNMDA',' AMPA')
name_order = list(models.values())
name_order = sorted(name_order,key=lambda x:(float(gmax_value_regex.match(x).group(0)),'AMPA' not in x))

grouping = {'_noNMDA'}
remove_v = True



recurstion old ,  3000
start job
start job


In [2]:
c = CTWHandler()
c.add_data(models_pathes)


EOFError: Ran out of input
 entropy_data\Rat_L5b_PC_2_Hay_0-4_CTW\ID_0_512971_Rat_L5b_PC_2_Hay_0-4_0.pkl